In [31]:
# some example data
import pandas as pd
from typing import Union
from statsmodels.tsa.vector_ar.var_model import VARResultsWrapper
from statsmodels.tsa.api import VAR

from statsmodels.tsa.base.datetools import dates_from_str
import numpy as np

In [32]:
import yfinance as yf

# 2023.TW 燁輝
# 2002.TW 中鋼
# 2006.TW 中和鋼鐵
# 2007.TW 燁興
date = '2016-01-01'
stock_no = '2002.TW'

stock = yf.Ticker(stock_no)
data = stock.history(start=date)

data#.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2016-01-04 00:00:00+08:00,13.274090,13.274090,12.905365,12.905365,24549000,0.00,0.0
2016-01-05 00:00:00+08:00,12.905366,13.163474,12.757876,12.794749,27544000,0.00,0.0
2016-01-06 00:00:00+08:00,12.831620,12.905365,12.757874,12.757874,18716000,0.00,0.0
2016-01-07 00:00:00+08:00,12.757875,12.905365,12.573512,12.868493,24452000,0.00,0.0
2016-01-08 00:00:00+08:00,12.757873,12.942235,12.647256,12.831618,20183000,0.00,0.0
...,...,...,...,...,...,...,...
2024-07-19 00:00:00+08:00,23.450001,23.500000,23.100000,23.350000,22809555,0.00,0.0
2024-07-22 00:00:00+08:00,23.250000,23.299999,23.000000,23.100000,26506583,0.00,0.0
2024-07-23 00:00:00+08:00,23.100000,23.250000,23.100000,23.150000,15851825,0.00,0.0


In [7]:
#data = pd.read_csv(dir+"National Stock Exchange/infy_stock.csv")

In [58]:
# """
# 要把 dataframe 的格式清理成以下格式
# dates_from_str 要放入 index
# 剩下的資料只能保留 feature
# """
# # statsmodels的範例資料
# import statsmodels.api as sm
# mdata = sm.datasets.macrodata.load_pandas().data
# mdata
# dates = mdata[['year', 'quarter']].astype(int).astype(str)
# quarterly = dates["year"] + "Q" + dates["quarter"]
# quarterly = dates_from_str(quarterly)
# mdata.index = pd.DatetimeIndex(quarterly)
# data = mdata.drop(columns=["year","quarter"])
# data

# # KAGGLE 範例資料 1 
# dir = "C:/Users/foresight_User/Desktop/公司文件/9.產品/AIUPS/"
# data = pd.read_csv(dir+"National Stock Exchange/infy_stock.csv")
# data.index =pd.DatetimeIndex(dates_from_str(data["Date"]))
# data = data.drop(columns=["Date","Symbol","Series"])
# data

# # KAGGLE 範例資料 2
# data = pd.read_csv(dir+"National Stock Exchange/nifty_it_index.csv")
# data.index =pd.DatetimeIndex(dates_from_str(data["Date"]))
# data = data.drop(columns=["Date"])
# data

# # KAGGLE 範例資料 3
# data = pd.read_csv(dir+"National Stock Exchange/tcs_stock.csv")
# data.index =pd.DatetimeIndex(dates_from_str(data["Date"]))
# data = data.drop(columns=["Date","Symbol","Series"])
# data

In [33]:
def time_series_format_preprocessing(df:pd.DataFrame, datetime_col:str):
    #
    # YS(年初), MS(月初), W(周), D(日), H(小時), T(分鐘), S(秒),
    #
    # df = df.copy()
    # df[datetime_col] = pd.to_datetime(df[datetime_col], format='%Y-%m-%d %H:%M:%S')
    # df = df.set_index(datetime_col)
    # df = df.asfreq(interval)
    # return df if set_index_flag else df.reset_index(drop=False)
    df.index =pd.DatetimeIndex(dates_from_str(data[datetime_col]))
    df = df.drop(columns=[datetime_col])
    """
    Imputes missing values in a time series DataFrame using the specified function.

    Args:
        df (pd.DataFrame): The input DataFrame containing the time series data.
        func (str): The imputation function to be applied. Supported options are:
                    - 'forward_fill': Forward fill missing values.
                    - 'backward_fill': Backward fill missing values.
                    - 'moving_average': Impute missing values using moving average.
                    - 'interpolation': Perform linear interpolation to fill missing values.
        col_name (str): The name of the column to impute missing values.

    Returns:
        pd.DataFrame: The DataFrame with missing values imputed based on the specified function.
    """
    return df


def time_series_impute_missing_value(df, func, col_name):
    """
    Imputes missing values in a time series DataFrame using the specified function.

    Args:
        df (pd.DataFrame): The input DataFrame containing the time series data.
        func (str): The imputation function to be applied. Supported options are:
                    - 'forward_fill': Forward fill missing values.
                    - 'backward_fill': Backward fill missing values.
                    - 'moving_average': Impute missing values using moving average.
                    - 'interpolation': Perform linear interpolation to fill missing values.
        col_name (str): The name of the column to impute missing values.

    Returns:
        pd.DataFrame: The DataFrame with missing values imputed based on the specified function.
    """
    df = df.copy()
    if func == 'forward_fill':
        df[col_name].ffill(inplace=True)
    elif func == 'backward_fill':
        df[col_name].bfill(inplace=True)
    elif func == 'moving_average':
        df[col_name].fillna(df[col_name].rolling(window=3, min_periods=1).mean(), inplace=True)
    elif func == 'interpolation':
        df[col_name].interpolate(inplace=True)
    else:
        raise ValueError(f"Unsupported imputation function: {func}")

    return df

In [34]:

# ==== 這邊不要動 =====

# """ 
# 這是套件設定的
# trend : str {"n", "c", "ct", "ctt"}
#     * "n" - no deterministic terms
#     * "c" - constant term
#     * "ct" - constant and linear term
#     * "ctt" - constant, linear, and quadratic term

# maxlags 不可以超過 max_estimable 的值
# maxlags 為模型擬合最大數值
# statemodel有設定條件，已經寫在下述的程式
# 使用者要調整低於 maxlags
# """
# max_estimable = cal_maxLag(data)
# # ==== 這邊不要動 =====

# print(" maxlags 要小於等於: ", max_estimable)

# if maxlags == "auto":
#     maxlags = max_estimable
# if maxlags > max_estimable:
#     raise Exception(" maxlags 要小於等於: ", max_estimable)


# """
# ic 為評估模型的好壞
# ic = {'aic', 'fpe', 'hqic', 'bic', None}
# Information criterion to use for VAR order selection.
# aic : Akaike
# fpe : Final prediction error
# hqic : Hannan-Quinn
# bic : Bayesian a.k.a. Schwarz
# """



In [35]:
def cal_maxLag(data:pd.DataFrame):
    n_totobs = len(data)
    ntrend = 1 #len(trend) if trend.startswith("c") else 0
    neqs = data.shape[1]
    max_estimable = (n_totobs - neqs - ntrend) // (1 + neqs)
    if max_estimable > 1:
        return max_estimable
    else:
        return 1

In [36]:

def vectorAutoregression(data:pd.DataFrame,maxlags:Union[int,str]="auto",ic:str=None):
    model = VAR(data)
    # ==== 這邊不要動 =====

    """ 
    這是套件設定的
    trend : str {"n", "c", "ct", "ctt"}
        * "n" - no deterministic terms
        * "c" - constant term
        * "ct" - constant and linear term
        * "ctt" - constant, linear, and quadratic term

    maxlags 不可以超過 max_estimable 的值
    maxlags 為模型擬合最大數值
    statemodel有設定條件，已經寫在下述的程式
    使用者要調整低於 maxlags
    """
    max_estimable = cal_maxLag(data)
    # ==== 這邊不要動 =====

    print(" maxlags 要小於等於: ", max_estimable)

    if maxlags == "auto":
        maxlags = max_estimable
    if maxlags > max_estimable:
        raise Exception(" maxlags 要小於等於: ", max_estimable)


    """
    ic 為評估模型的好壞
    ic = {'aic', 'fpe', 'hqic', 'bic', None}
    Information criterion to use for VAR order selection.
    aic : Akaike
    fpe : Final prediction error
    hqic : Hannan-Quinn
    bic : Bayesian a.k.a. Schwarz
    """

    results = model.fit(maxlags=maxlags, ic=ic)
    print(f"在最大 lag 數目為 {max_estimable} 的情況下，VAR 找出的最佳 lag 為: ",results.k_ar)
    return results

In [37]:
def vectorAutoregressionRelationship(results:VARResultsWrapper,target:str,pvalue_threshold:float=0.05):
    # target  客人關心的 Y 是甚麼，Y 會包含在 results 中

    coef_df = results.params[target]
    pvalues_df = results.pvalues[target]

    # 合并系数和p值
    summary = pd.concat([coef_df, pvalues_df], axis=1)
    summary.columns = ['coef', 'pvalue']
    summary = summary.drop(index="const").reset_index()
    summary_index = summary["index"].str.split(".", expand=True).rename(columns={0:"time_lag",1:"feature"})
    if summary.empty:
        return {"info":"there is no results found from VAR"}
    else:
        summary_index["time_lag"] = summary_index["time_lag"].str.replace("L","").astype(int)
        summary = pd.concat([summary_index,summary],axis=1).drop(columns="index")
        summary = summary[summary["pvalue"]<pvalue_threshold].reset_index(drop=True)
        return summary

In [38]:



# 定義聚合函數和相應的列名
operator_list = ['first', 'last', 'mean','prod','median','max','min']#'std','max','count','sum',"min","prod","median","var","sem","skew","cumsum","cumprod"]


aggregation_functions = {}
for i in data.columns:
    aggregation_functions[i]=operator_list

# 按週重採樣並應用聚合
weekly_data = data.resample('W').agg(aggregation_functions)

# 自定義列名

monthly_data = data.resample('M').agg(aggregation_functions)
yearly_data = data.resample('Y').agg(aggregation_functions)

weekly_data.columns = ['_'.join(col).strip() for col in weekly_data.columns.values]
monthly_data.columns = ['_'.join(col).strip() for col in monthly_data.columns.values]
yearly_data.columns = ['_'.join(col).strip() for col in yearly_data.columns.values]


C:\Users\foresight_User\AppData\Local\Temp\ipykernel_17380\1254553657.py:14: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_data = data.resample('M').agg(aggregation_functions)
C:\Users\foresight_User\AppData\Local\Temp\ipykernel_17380\1254553657.py:15: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  yearly_data = data.resample('Y').agg(aggregation_functions)


In [39]:

def remove_collinearity(data,remain:list=None): #condition
    from statsmodels.stats.outliers_influence import variance_inflation_factor
    if remain != None:
        remained_data = data.loc[:,remain]
        data = data.drop(columns=remain)
    #data = data.fillna(method='ffill').fillna(method='bfill')
    data = data.replace([np.inf, -np.inf], np.nan)
    data = data.ffill().bfill() 
    vif_data = pd.DataFrame()
    vif_data["feature"] = data.columns
    vif_data["VIF"] = [variance_inflation_factor(data.values, i) for i in range(len(data.columns))]


    # 去除VIF為NAN或INF
    vif_data = vif_data.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
    data = data[vif_data["feature"].to_list()]

    # 移除高相關的 feature 
    # 保留低相關的 feature
    coefficient_threshold = 0.3
    correlation_matrix = data.corr()
    low_corr_vars = np.where(np.abs(correlation_matrix) < coefficient_threshold)
    #print(correlation_matrix)
    low_corr_vars = [(correlation_matrix.index[x], correlation_matrix.columns[y]) for x, y in zip(*low_corr_vars) if x != y and x > y]

    # Remove one of each highly correlated pair
    # if condition == "intersection":
    #     sets =  [set(pair) for pair in low_corr_vars]
    #     intersection = set.intersection(*sets)
    #     remaining_list = list(intersection)
    # if condition == "union":
    unique_elements = set(element for pair in low_corr_vars for element in pair)
    remaining_list = list(unique_elements)
    
    data = data[remaining_list]
    if remain != None:
        data = pd.concat([data,remained_data],axis=1)
    # impute nan
    #data = data.fillna(method='ffill').fillna(method='bfill')
    return data

In [40]:
"""
讓使用者決定好奇的target是甚麼，套件會自動找出所有跟target相關的不同time lag的時間變相
maxlags 會影響最終最佳推薦的結果，也得慎選，但設定上不可以超過 cal_maxLag 輸出的值
"""
# #data = pd.read_csv(dir+"National Stock Exchange/tcs_stock.csv").drop(columns=["Symbol","Series"])
# data = time_series_format_preprocessing(data,"Date")
# data = time_series_impute_missing_value(data, 'interpolation', "Volume")

# 如果有缺失值，可以选择填充或删除


# 
# 
# 

for d in [data,weekly_data,monthly_data,yearly_data]:
    for i in d.columns:
        print(target)
        target = i #"Open_first"#"Trades"
        clean_data = remove_collinearity(data=d,remain=[target])
        maxlags = cal_maxLag(clean_data) # USER 不能指定超過這個的LAG，LAG值也要大於0
        print(maxlags)
        results = vectorAutoregression(clean_data,maxlags=3,ic=None)
        pvalue_threshold = 0.05
        VAR_relationship = vectorAutoregressionRelationship(results=results,target=target,pvalue_threshold=pvalue_threshold)
        break

Dividends_max
296
 maxlags 要小於等於:  296
在最大 lag 數目為 296 的情況下，VAR 找出的最佳 lag 為:  3
Open


c:\Users\foresight_User\anaconda3\envs\Python3.9.13\lib\site-packages\statsmodels\regression\linear_model.py:1785: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss
c:\Users\foresight_User\anaconda3\envs\Python3.9.13\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\foresight_User\anaconda3\envs\Python3.9.13\lib\site-packages\statsmodels\regression\linear_model.py:1783: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


9
 maxlags 要小於等於:  9


LinAlgError: SVD did not converge in Linear Least Squares

In [42]:
target

'Open_first'

In [30]:
df

,time_lag,feature,coef,pvalue
0,1,Open_min,3.860940e-02,0.000000
1,1,Close_last,2.649091e-02,0.000000
2,1,Open_max,-2.781502e-02,0.000000
3,1,Close_min,3.438232e-02,0.000000
4,1,Volume_last,-4.559469e-08,0.000000
5,1,Volume_mean,-4.983963e-08,0.000000
6,1,Low_min,3.127998e-02,0.000000
7,1,Low_median,-2.396077e-03,0.000093
8,1,Volume_first,-2.419460e-08,0.000000
9,1,Close_first,-3.561709e-02,0.000000


In [29]:
df.loc[2,"time_lag"]
df.loc[2,"feature"]#["feature"]

'Open_max'

In [ ]:

df = pd.DataFrame(data)
df['TIME'] = pd.to_datetime(df['TIME'])

# 添加 X_1 的 time lag 1 和 2
df['X_1_lag1'] = df['X_1'].shift(1)
df['X_1_lag2'] = df['X_1'].shift(2)

# 添加 X_2 的 time lag 4
df['X_2_lag4'] = df['X_2'].shift(4)

# 將 TIME 設為索引（可選）
df.set_index('TIME', inplace=True)

# 移除包含 NaN 值的行
#df.dropna(inplace=True)

print(df)


In [24]:
coef_df = results.params[target]
pvalues_df = results.pvalues[target]

# 合并系数和p值
summary = pd.concat([coef_df, pvalues_df], axis=1)
summary.columns = ['coef', 'pvalue']
summary = summary.drop(index="const").reset_index()

In [5]:
VAR_relationship

NameError: name 'VAR_relationship' is not defined

In [4]:
results.summary()

NameError: name 'results' is not defined

In [27]:
data


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2016-01-04 00:00:00+08:00,13.274089,13.274089,12.905364,12.905364,24549000,0.00,0.0
2016-01-05 00:00:00+08:00,12.905366,13.163474,12.757876,12.794749,27544000,0.00,0.0
2016-01-06 00:00:00+08:00,12.831618,12.905363,12.757873,12.757873,18716000,0.00,0.0
2016-01-07 00:00:00+08:00,12.757874,12.905364,12.573511,12.868492,24452000,0.00,0.0
2016-01-08 00:00:00+08:00,12.757874,12.942236,12.647257,12.831619,20183000,0.00,0.0
...,...,...,...,...,...,...,...
2024-07-19 00:00:00+08:00,23.450001,23.500000,23.100000,23.350000,22809555,0.00,0.0
2024-07-22 00:00:00+08:00,23.250000,23.299999,23.000000,23.100000,26506583,0.00,0.0
2024-07-23 00:00:00+08:00,23.100000,23.250000,23.100000,23.150000,15851825,0.00,0.0


In [28]:
"""
生成VAR關係公式
"""
"""
可有可無，看需求
"""
formula_parts = []
for index, row in VAR_relationship.iterrows():
    coef = row['coef']
    if coef < 0:
        term = f"- {-coef:.3f} * {row['feature']}(t-{row['time_lag']})"
    else:
        term = f"{coef:.3f} * {row['feature']}(t-{row['time_lag']})"
    formula_parts.append(term)

formula = " + ".join(formula_parts).replace("+ -", "- ")
target_formula = f"f{target}(t) = {formula}"

# 打印公式
print(target_formula)

fVolume_min(t) = 1.100 * Volume_min(t-1) -  210634.048 * High_first(t-1) + 166969.493 * Low_last(t-1) + 1.491 * Volume_median(t-1) + 0.310 * Volume_first(t-1) -  199683.404 * Close_max(t-1) -  1.684 * Volume_mean(t-1) + 155007.475 * High_last(t-1) -  195455.261 * Open_max(t-1) -  248780.777 * Close_first(t-1) -  32629.428 * Open_median(t-1) + 0.157 * Volume_max(t-1) + 3534.103 * Low_mean(t-1)


In [29]:
def time_series_format_preprocessing(df: pd.DataFrame, interval, datetime_col, set_index_flag=False):
    #
    # YS(年初), MS(月初), W(周), D(日), H(小時), T(分鐘), S(秒),
    #
    df = df.copy()
    df[datetime_col] = pd.to_datetime(df[datetime_col], format='%Y-%m-%d %H:%M:%S')
    df = df.dropna(subset=[datetime_col])
    # 方法1
    # df = df.set_index(datetime_col)
    # df = df[~df.index.duplicated(keep='first')]
    # df = df.asfreq(interval)
    # return df if set_index_flag else df.reset_index(drop=False)

    # 方法2
    df = df.set_index(datetime_col, append=True)
    def asfreq(df, freq):
        # 防止索引名稱為null
        names = []
        for i, name in enumerate(df.index.names):
            if name is None:
                names.append(f"level_{i}")
            else:
                names.append(name)
        df.index.names = names
        # 重設第一個索引層級
        level_to_reset = df.index.names[0]
        df_reset = df.reset_index(level=level_to_reset)
        df_reset = df_reset[~df_reset.index.duplicated(keep='first')]
        # 重新設置頻率
        df_resampled = df_reset.asfreq(freq)
        df_resampled.reset_index(inplace=True) # 會導致freq設定消失
        # 設置新的索引
        for i, name in enumerate(df.index.names):
            df_resampled.set_index(name, inplace=True, append=True if i > 0 else False)
        return df_resampled

    df = df[~df.index.duplicated(keep='first')]
    df = asfreq(df, interval)

    df.index.levels[1].freq = interval
    return df if set_index_flag else df.reset_index(drop=False, level=df.index.names[1]) # 0是原索引，1是時間索引

In [2]:
time_series_format_preprocessing(data.reset_index(),"1W",datetime_col="Date")

NameError: name 'time_series_format_preprocessing' is not defined

In [3]:
data.asfreq("1W")

NameError: name 'data' is not defined